#### Stage 2
# Flow Data - Spatial Overview - Anomalies
with folium maps

##### Generating Flow Maps Showing Anomalies
This notebook contains the code to generate visualization **number 5** from **anomFlowdf.csv** similarly to **visualization 1**.
The lines now show the anomalies of Wednesday with respect to the average movements of Monday, Tuesday and Thursday and are colour-coded according to the type of anomaly: green if there is more movement on Wednesday, charcoal grey if there is less.
The dataset is filtered and aggregated according to the chosen hours of the day and the minimal anomaly.
The lines' widths are calculated using the anomaly within the chosen mode and are again either normalized to the chosen width factor or the averaged values for Monday, Tuesday and Thursday in case "relativ" is checked in the widget.
Tooltips for the cell polygons, now coloured according to the chosen mode, are added as for visualization number 1 while the tooltips for the flow lines additionally include the ratio of movements on Wednesday to the average movement numbers on the other three days.

##### Versions of the used packages:
- pandas: 0.24.2
- numpy: 1.16.4
- folium: 0.11.0
- ipywidgets: 7.5.1

In [1]:
import pandas as pd
from ast import literal_eval
import folium
import numpy as np
import ipywidgets as pyw

In [2]:
centroidBounds = [[[48.04821822889212, 11.67438639612782]],[[48.39922017377597, 11.75558830832081]]]
mapbox_access_token = r'MAPBOX_ACCESS_TOKEN'
tileset_ID_str = 'streets-v11'
# tileset_ID_str = 'outdoors-v11'
# tileset_ID_str = 'light-v10'
# tileset_ID_str = 'dark-v10'
# tileset_ID_str = 'satellite-v9'
# tileset_ID_str = 'satellite-streets-v11'
tilesize_pixels = '256'
#Tiles = 'OpenStreetMap'
#Tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"
Tiles = f"https://api.mapbox.com/styles/v1/klavere/ckd8wc3hf06we1imiypydrqcf/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"
MapboxAttribution = '<a href="https://www.mapbox.com/about/maps/">© Mapbox</a> | <a href="http://www.openstreetmap.org/about/">© OpenStreetMap</a> | <a href="https://www.mapbox.com/map-feedback/#/-74.5/40/10">Improve this map</a>'

In [3]:
def csvtodf_SC(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=';',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def csvtodf_C(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=',',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def getCoordsBack(coords):
    return literal_eval(coords);

In [4]:
celldf = csvtodf_SC('celldf')
celldf['polyCoords'] = celldf.apply(lambda row: getCoordsBack(row.polyCoords), axis = 1)
celldf['centroidCoords'] = celldf.apply(lambda row: getCoordsBack(row.centroidCoords), axis = 1)
celldf = celldf.reindex(columns = ['cellID', 'polyCoords', 'centroidCoords'])
celldf.head()

,cellID,polyCoords,centroidCoords
0,1,"[[48.14605999999999, 11.510794], [48.146254999...","[48.15396752884293, 11.51032914199828]"
1,2,"[[48.23591199999999, 11.635515], [48.237070000...","[48.22263647015427, 11.62774480807693]"
2,3,"[[48.14900599999999, 11.696217], [48.153606000...","[48.13564057367491, 11.70093894926043]"
3,4,"[[48.12879600000001, 11.541556], [48.125282, 1...","[48.13194111905815, 11.54797042679206]"
4,5,"[[48.18833000000001, 11.615821], [48.193665, 1...","[48.19669108307331, 11.6113147037113]"


In [5]:
flowAnomdf = csvtodf_SC('anomFlowdf')
flowAnomdf['flowCoords'] = flowAnomdf.apply(lambda row: getCoordsBack(row.flowCoords), axis = 1)
flowAnomdf.head()

,flowID,flowCoords,hour,moves,privat,public,Rail,UBahn,Tram,Bus,...,UBahnMDD,TramMDD,BusMDD,movesAnom,privatAnom,publicAnom,RailAnom,UBahnAnom,TramAnom,BusAnom
0,100_101,"[[48.22013108151531, 11.52767333862167], [48.3...",18,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,100_101,"[[48.22013108151531, 11.52767333862167], [48.3...",19,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,100_102,"[[48.22013108151531, 11.52767333862167], [48.1...",2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100_102,"[[48.22013108151531, 11.52767333862167], [48.1...",4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100_102,"[[48.22013108151531, 11.52767333862167], [48.1...",7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0


## map flow anomalies

maps flow anomalies absolute or wrt mean at a normal work day for the respective segment

In [6]:
days = ['', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dayTypes = ['', 'MonThu', 'Fri', 'Weekend']

modes = ['moves', 'privat', 'public', 'Rail', 'UBahn', 'Tram', 'Bus']
modesAnom = ['']*7
for i in range(0,7):
    modesAnom[i] = modes[i]+'Anom'

modesMDD = ['']*7
for i in range(0,7):
    modesMDD[i] = modes[i]+'MDD'

modesColor = {'privat':'#999999',#'#BFBFBF',
              'Rail':'#4daf4a',
              'UBahn':'#377eb8',
              'Tram':'#e41a1c',
              'Bus':'#984ea3',
              'public':'#ff7f00', # '#ff7f00',
              'moves':'#a65628'} # '#FFFFFF'}
# including ColorBrewer Colors from: https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=9
PosNegCol = {True:'#1a9641',
             False:'#404040'}

d = {'flowCoords':['first']}
# modesAggMean = dict((key, ['mean']) for key in modes+modesMDD+modesAnom)
modesAggSum = dict((key, ['sum']) for key in modes+modesMDD+modesAnom)
d.update(modesAggSum)

In [7]:
def mapFlowAnomalies(mode, hours=(18,24), minAnom=20, widthfactor=100, relativ=False):
    
    modeAnom = mode+'Anom'
    modeMDD = mode+'MDD'
    
    mapdf = flowAnomdf[(flowAnomdf.hour.between(hours[0]-1,hours[1],inclusive=False))]
    mapdf = mapdf.groupby(['flowID']).agg(d).copy().reset_index()
    mapdf.columns = mapdf.columns.get_level_values(0)

    mapdf = mapdf[(abs(mapdf[modeAnom]) >= minAnom)]# &(mapdf[mode]>=5)&(mapdf[modeMDD]>=5)]
    
    # initialize map with cell polygons
    m = folium.Map(tiles = Tiles,
                   attr = MapboxAttribution)
    m.fit_bounds(centroidBounds)
    for index, row in celldf.iterrows():
        line_i = folium.Polygon(locations=row.polyCoords,
                                color = modesColor[mode],# color = '#ff7f00',#'#FBF9F7',
                                weight = 1,
                                fill = True,
                                opacity = 0.5,
                                fill_opacity = 0.02,
                                tooltip = 'Cell'+str(row.cellID)).add_to(m)
    
    
    # put anomalies on map
    # M = 2*np.mean(abs(mapdf[modeAnom])) # mean for linewidths if relative=False //doesn't make sense here

    for index, row in mapdf.iterrows():
        absAnom = round(abs(row[modeAnom]))
        MDD = round(row[modeMDD])
        if (row[modeAnom] > 0):
            color = PosNegCol[True]#'#1a9641'
            side = 'more'
        else:
            color = PosNegCol[False]#'#404040'
            side = 'less'
        if (relativ):
            weight = (absAnom/MDD)*5
        else:
            weight = absAnom/(widthfactor+1)
        tooltip = str(absAnom)+' moves '+side+'<br>'+str(np.round(row[mode]/MDD, decimals=1))+' times the moves on a normal weekday <br>mode = '+str(mode)+'<br>FlowID '+str(row.flowID)
        line_i = folium.PolyLine(locations = row.flowCoords,
                                 color = color,
                                 weight = weight,
                                 opacity = 0.6,
                                 tooltip = tooltip)
        m.add_child(line_i)
    return m;

In [8]:
range_slider_h = pyw.IntRangeSlider(
    value=[0,24],
    min=0, max=24, step=1,
    description='Time of day')

range_slider_mA = pyw.IntSlider(
    value=10,
    min=10, max=100, step=5,
    description='Anom. min')

range_slider_lw = pyw.IntSlider(
    value=100,
    min=0, max=500, step=10,
    description='Width (abs)')

In [9]:
pyw.interact_manual(mapFlowAnomalies,
                    mode = modes,
                    hours = range_slider_h,
                    minAnom = range_slider_mA,
                    widthfactor = range_slider_lw)

interactive(children=(Dropdown(description='mode', options=('moves', 'privat', 'public', 'Rail', 'UBahn', 'Tra…

<function __main__.mapFlowAnomalies(mode, hours=(18, 24), minAnom=20, widthfactor=100, relativ=False)>